Inspired by https://climada-python.readthedocs.io/en/stable/tutorial/climada_util_earth_engine.html

**Connect to Google Earth Engine (GEE) API**

In [1]:
# Authorize access to your Earth Engine account
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=41sKhsZD2toRt2LLox3mX4DqCQCPJ5oP_feRjypx3UM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5jWqTgxs3ZaaJnnlEhtB4kaErnsthF0oBix4GtmJrn49KDVRwXfg8

Successfully saved authorization token.


In [2]:
import webbrowser
import ee
ee.Initialize()

**Define Area of Interest**

In [18]:
list_dunhuang = list([(94.4, 40.45),(94.4, 40),(94.95, 40),(94.95, 40.45),(94.4, 40.45)])
area_dunhuang = ee.Geometry.Polygon(list_dunhuang)
time_range_dunhuang_1=['2016-08-01', '2016-08-31']
time_range_dunhuang_2=['2020-08-01', '2020-08-31']
collection_dunhuang = ('COPERNICUS/S2')

**Obtain Image**

In [4]:
def obtain_image_median(collection, time_range, area):
    """ Selection of median from a collection of images in the Earth Engine library
    See also: https://developers.google.com/earth-engine/reducers_image_collection

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        image_median (ee.image.Image)
     """
    collection = ee.ImageCollection(collection)

    ## Filter by time range and location
    collection_time = collection.filterDate(time_range[0], time_range[1])
    image_area = collection_time.filterBounds(area)
    image_median = image_area.median()
    return image_median

def obtain_image_sentinel(collection, time_range, area):
    """ Selection of median, cloud-free image from a collection of images in the Sentinel 2 dataset
    See also: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

    Parameters:
        collection (): name of the collection
        time_range (['YYYY-MT-DY','YYYY-MT-DY']): must be inside the available data
        area (ee.geometry.Geometry): area of interest

    Returns:
        sentinel_median (ee.image.Image)
     """
#First, method to remove cloud from the image
    def maskclouds(image):
        band_qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        cirrus_mask = ee.Number(2).pow(11).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0) and(
            band_qa.bitwiseAnd(cirrus_mask).eq(0))
        return image.updateMask(mask).divide(10000)

    sentinel_filtered = (ee.ImageCollection(collection).
                         filterBounds(area).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).
                         map(maskclouds))

    sentinel_median = sentinel_filtered.median()
    return sentinel_median

In [19]:
Dunhuang_2016_Summer = obtain_image_sentinel(collection_dunhuang, time_range_dunhuang_1, area_dunhuang)
Dunhuang_2020_Summer = obtain_image_sentinel(collection_dunhuang, time_range_dunhuang_2, area_dunhuang)

# Selection of specific bands from an image
band_2016 = Dunhuang_2016_Summer.select(['B4','B3','B2'])
band_2020 = Dunhuang_2020_Summer.select(['B4','B3','B2'])


print(Dunhuang_2016_Summer.getInfo())
print(type(band_2020))

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1,

**Download Image**


In [20]:
def get_region(geom):
    """Get the region of a given geometry, needed for exporting tasks.

    Parameters:
        geom (ee.Geometry, ee.Feature, ee.Image): region of interest

    Returns:
        region (list)
    """
    if isinstance(geom, ee.Geometry):
        region = geom.getInfo()["coordinates"]
    elif isinstance(geom, ee.Feature, ee.Image):
        region = geom.geometry().getInfo()["coordinates"]
    elif isinstance(geom, list):
        condition = all([isinstance(item) == list for item in geom])
        if condition:
            region = geom
    return region
  
region_dunhuang = get_region(area_dunhuang)

In [24]:
def get_url(name, image, scale, region):
    """It will open and download automatically a zip folder containing Geotiff data of 'image'.
    If additional parameters are needed, see also:
    https://github.com/google/earthengine-api/blob/master/python/ee/image.py

    Parameters:
        name (str): name of the created folder
        image (ee.image.Image): image to export
        scale (int): resolution of export in meters (e.g: 30 for Landsat)
        region (list): region of interest

    Returns:
        path (str)
     """
    path = image.getDownloadURL({
        'name':(name),
        'scale': scale,
        'region':(region)
        })

    # webbrowser.open_new_tab(path)
    return path



url_dunhuang_summer_2016 = get_url('Dunhuang_Sentinel_20160801_20160831', band_2016, 50, region_dunhuang)
url_dunhuang_summer_2020 = get_url('Dunhuang_Sentinel_20200801_20200831', band_2020, 50, region_dunhuang)

#For the example of Zürich, due to size, it doesn't work on Jupyter Notebook but it works on Python
#url_zurich = get_url('sentinel', zurich_band, 10, region_zurich)
print(url_dunhuang_summer_2016, url_dunhuang_summer_2020)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b91fe94b0daa45496f864d5e84ab58ce-43fe9dd7d97974e4f29ff1fd8e381e80:getPixels https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0c29459778e3f34ef2a85f4977c37ca9-55fc16a4b5ee884c5f989ec93f4f29ba:getPixels


**Simple Visualization**
